# LGBM VMD Model development

In [1]:
import pandas as pd
import seaborn as sns
import openpyxl
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle

In [2]:
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna
from vmdpy import VMD

In [3]:
from Functions.helper_functions import * 

In [4]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
np.random.seed(42)

# Data

In [6]:
train_wp1 = pd.read_csv('Data/Preprocessing/WP1_train_preprocessed.csv', sep=',')
train_wp2 = pd.read_csv('Data/Preprocessing/WP2_train_preprocessed.csv', sep=',')
train_wp3 = pd.read_csv('Data/Preprocessing/WP3_train_preprocessed.csv', sep=',')
train_wp4 = pd.read_csv('Data/Preprocessing/WP4_train_preprocessed.csv', sep=',')
train_wp5 = pd.read_csv('Data/Preprocessing/WP5_train_preprocessed.csv', sep=',')
train_wp6 = pd.read_csv('Data/Preprocessing/WP6_train_preprocessed.csv', sep=',')

In [7]:
test_wp1 = pd.read_csv('Data/Preprocessing/WP1_test_preprocessed.csv', sep=',')
test_wp2 = pd.read_csv('Data/Preprocessing/WP2_test_preprocessed.csv', sep=',')
test_wp3 = pd.read_csv('Data/Preprocessing/WP3_test_preprocessed.csv', sep=',')
test_wp4 = pd.read_csv('Data/Preprocessing/WP4_test_preprocessed.csv', sep=',')
test_wp5 = pd.read_csv('Data/Preprocessing/WP5_test_preprocessed.csv', sep=',')
test_wp6 = pd.read_csv('Data/Preprocessing/WP6_test_preprocessed.csv', sep=',')
test_dates = pd.read_csv('Data/Initial/test.csv', sep=',').date.values

In [8]:
to_drop = ['date','wd','forecast_time', 'forecast', "forecast_dist"]

In [9]:
u_to_drop = [
    'u_T_1', 'u_T_2', 'u_T_3', 'u_T_4', 'u_T_5', 'u_T_6', 
    'u_T_2_mean', 'u_T_3_mean', 'u_T_4_mean', 'u_T_5_mean', 'u_T_6_mean', 'u_T_7_mean',
    'u_T_8_mean', 'u_T_9_mean', 'u_T_10_mean', 'u_T_11_mean', 'u_T_12_mean','u_T_24_mean',
    'u_T_2_std', 'u_T_4_std', 'u_T_5_std', 'u_T_6_std',
    'u_T_2_median', 'u_T_3_median', 'u_T_4_median', 'u_T_5_median', 'u_T_6_median', 'u_T_12_median','u_T_24_median', 'u_T_36_median',
    'u_T_2_max', 'u_T_3_max', 'u_T_4_max', 'u_T_5_max', 'u_T_6_max', 'u_T_12_max',
    'u_T_2_min', 'u_T_3_min', 'u_T_4_min', 'u_T_5_min', 'u_T_6_min', 'u_T_12_min',
    'u2_T_1', 'u2_T_2', 'u2_T_3', 'u2_T_4', 'u2_T_5', 'u2_T_6', 
    'u2_T_2_mean', 'u2_T_3_mean', 'u2_T_4_mean', 'u2_T_5_mean', 'u2_T_6_mean', 'u2_T_7_mean',
    'u2_T_8_mean', 'u2_T_9_mean', 'u2_T_10_mean', 'u2_T_11_mean', 'u2_T_12_mean','u2_T_24_mean',
    'u2_T_2_std', 'u2_T_4_std', 'u2_T_5_std', 'u2_T_6_std', 'u2_T_24_std',
    'u2_T_2_median', 'u2_T_3_median', 'u2_T_4_median', 'u2_T_5_median', 'u2_T_6_median', 'u2_T_12_median',
    'u2_T_2_max','u2_T_3_max', 'u2_T_4_max','u2_T_5_max', 'u2_T_6_max', 'u2_T_12_max',
    'u2_T_2_min', 'u2_T_3_min', 'u2_T_4_min', 'u2_T_5_min', 'u2_T_6_min',
    'u2_T_12', 'u2_T_36_mean', 'u2_T_36_std', 'u2_T_24_median', 'u2_T_24_max',
    'u_T_36_mean','u_T_12','u_T_24_max','u2_T_36_median','u_T_24_min'
]
ws_to_drop = [
    'ws_T_1', 'ws_T_2', 'ws_T_3', 'ws_T_4', 'ws_T_5', 'ws_T_6', 'ws_T_7', 'ws_T_8', 'ws_T_10','ws_T_11', 'ws_T_12',
    'ws_T_2_mean', 'ws_T_3_mean', 'ws_T_4_mean', 'ws_T_5_mean', 'ws_T_6_mean', 'ws_T_7_mean', 'ws_T_8_mean', 'ws_T_9_mean', 
    'ws_T_10_mean', 'ws_T_11_mean', 'ws_T_12_mean', 'ws_T_24_mean', 
    'ws_T_2_std', 'ws_T_3_std', 'ws_T_4_std', 'ws_T_5_std', 
    'ws_T_2_median', 'ws_T_3_median', 'ws_T_4_median', 'ws_T_5_median', 'ws_T_6_median',
    'ws_T_12_median', 'ws_T_24_median', 'ws_T_36_median',
    'ws_T_2_max', 'ws_T_3_max', 'ws_T_4_max', 'ws_T_5_max','ws_T_6_max', 'ws_T_12_max',
     'ws_T_2_min', 'ws_T_3_min', 'ws_T_4_min', 'ws_T_5_min', 'ws_T_6_min', 'ws_T_12_min','ws_T_24_max','ws_T_24_min'
]

v_to_drop = [
    'v_T_1', 'v_T_2', 'v_T_3', 'v_T_4', 'v_T_5', 'v_T_6', 
    'v_T_2_mean', 'v_T_3_mean', 'v_T_4_mean', 'v_T_5_mean', 'v_T_6_mean', 'v_T_7_mean',
    'v_T_8_mean', 'v_T_9_mean', 'v_T_10_mean', 'v_T_11_mean', 'v_T_12_mean', 'v_T_24_mean','v_T_36_mean',
    'v_T_3_std', 'v_T_4_std', 'v_T_5_std','v_T_6_std','v_T_24_std', 'v_T_36_median',
    'v_T_2_median', 'v_T_3_median', 'v_T_4_median', 'v_T_5_median', 'v_T_6_median', 
    'v_T_2_max', 'v_T_3_max', 'v_T_4_max', 'v_T_5_max', 'v_T_6_max', 'v_T_12_max', 
    'v_T_2_min', 'v_T_3_min', 'v_T_4_min', 'v_T_5_min', 'v_T_6_min', 'v_T_12_min', 
    'v_T_36_min', 'v_T_36', 'v_T_24_max',  'v_T_12_median', 'v_T_24_median',
]

wd_to_drop = [
    'coswd_1', 'coswd_2', 'coswd_3', 'coswd_4', 'coswd_5', 'coswd_6',
    'coswd_2_mean', 'coswd_3_mean', 'coswd_4_mean', 'coswd_5_mean', 'coswd_6_mean', 'coswd_7_mean', 
    'coswd_8_mean', 'coswd_9_mean', 'coswd_10_mean', 'coswd_11_mean', 'coswd_12_mean', 'coswd_24_mean', 
    'coswd_3_std', 'coswd_4_std','coswd_5_std','coswd_2_median', 'coswd_3_median','coswd_4_median', 
    'coswd_5_median', 'coswd_6_median', 'coswd_36_median', 'coswd_24_median', 'coswd_12_median',
    'coswd_2_max', 'coswd_3_max', 'coswd_4_max', 'coswd_5_max', 'coswd_6_max', 'coswd_12_max', 'coswd_24_max',
    'coswd_2_min', 'coswd_3_min', 'coswd_4_min', 'coswd_5_min', 'coswd_6_min', 'coswd_12_min', 'coswd_24_min',
    'ws_T_36_max', 'ws_T_36_min', 'coswd_12', 'coswd_24'
]

other_to_drop = [
    'cos_day', 'u', 'v'
]

feature_corr = u_to_drop+ws_to_drop+v_to_drop+wd_to_drop+other_to_drop
to_drop = feature_corr+to_drop

# LGBM functions

In [10]:
from lightgbm import LGBMRegressor

In [11]:
# def lgbm_cross_validation(X, y, params):
#     if params == None:
#         model = LGBMRegressor()
#     else:
#         model = LGBMRegressor(**params)

#     print('-----------LGBM CROSS VALIDATION BEGINNING-----------')
#     split = 10
#     kf = KFold(n_splits=split, shuffle=True)       
#     lgbm_rmse_scores = []
#     lgbm_mae_scores = []
#     i = 1
#     for (train_index, test_index) in kf.split(pd.DataFrame(X), pd.DataFrame(y)):
#         X_train, X_test = pd.DataFrame(X).iloc[train_index], pd.DataFrame(X).iloc[test_index]
#         Y_train, Y_test = pd.DataFrame(y).iloc[train_index],pd.DataFrame(y).iloc[test_index]

#         model.fit(X_train, Y_train, eval_set=[(X_test, Y_test)], verbose=100)

#         prediction = model.predict(X_test)
#         lgbm_rmse_scores.append(mean_squared_error(Y_test, prediction,squared=False))
#         lgbm_mae_scores.append(mean_absolute_error(Y_test, prediction))
        
#         print(show_evaluation(prediction, Y_test))
#         print(f'-------------------FOLD {i}-----------------')
#         i+=1

#     print('---------------CROSS VALIDATION COMPLETE-------------')
#     print('--------------------------RMSE-----------------------')
#     display_scores(lgbm_rmse_scores)
#     print('--------------------------MAE------------------------')
#     display_scores(lgbm_mae_scores)

In [12]:
def lgbm_scaled_cross_validation(X, y, params):
    if params == None:
#         model = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
        model = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor())])
    else:
#         model = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**params))])
        model = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor(**params))])

    print('-----------LGBM CROSS VALIDATION BEGINNING-----------')
    split = 10
    kf = KFold(n_splits=split, shuffle=True)       
    lgbm_rmse_scores = []
    lgbm_mae_scores = []
    i = 1
    for (train_index, test_index) in kf.split(pd.DataFrame(X), pd.DataFrame(y)):
        X_train, X_test = pd.DataFrame(X).iloc[train_index], pd.DataFrame(X).iloc[test_index]
        Y_train, Y_test = pd.DataFrame(y).iloc[train_index],pd.DataFrame(y).iloc[test_index]

        model.fit(X_train, Y_train)

        prediction = model.predict(X_test)
        lgbm_rmse_scores.append(mean_squared_error(Y_test, prediction,squared=False))
        lgbm_mae_scores.append(mean_absolute_error(Y_test, prediction))
        
        print(show_evaluation(prediction, Y_test))
        print(f'-------------------FOLD {i}-----------------')
        i+=1

    print('---------------CROSS VALIDATION COMPLETE-------------')
    print('--------------------------RMSE-----------------------')
    display_scores(lgbm_rmse_scores)
    print('--------------------------MAE------------------------')
    display_scores(lgbm_mae_scores)

In [13]:
def hyperparametrization(trial, train_x, test_x, train_y, test_y):
    param = {
        'metric': 'rmse', 
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'max_depth': trial.suggest_int("max_depth", 20, 100),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
    }
    
#     model = LGBMRegressor(**param)  
    model = Pipeline([('scaler', MinMaxScaler()),('ridge', LGBMRegressor(**param))])
#     model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    model.fit(train_x,train_y)
    
    preds = model.predict(test_x)
    
    rmse =  mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [14]:
def hyperparametrization_cv(trial, X, y):
    param = {
        'metric': 'rmse', 
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'max_depth': trial.suggest_int("max_depth", 20, 100),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
    }
    
    model = LGBMRegressor(**param)
    rmse = - cross_val_score(model, X, y, cv=3, scoring = 'neg_root_mean_squared_error').mean()
    
    return rmse

In [15]:
def vmd(y,k,alpha, tau,DC, init, tol):
    
    #Intrinsic mode generation
     #Empirical Mode Decomposition
    #. some sample parameters for VMD  
    alpha = 1       # moderate bandwidth constraint  
    tau = 0.           # noise-tolerance (no strict fidelity enforcement)  
    K = k              # k modes  
    DC = 0             # no DC part imposed  
    init = 1           # initialize omegas uniformly  
    tol = 1e-7
    u, u_hat, omega = VMD(y,alpha, tau, K, DC, init, tol)
    df_vmfs = pd.DataFrame()
    #Integration in the dataframe
    for num, imf in enumerate(u):
        #print('----Creating VMFwp{0} EMD columns----'.format(num+1))
        df_vmfs['IMFwp{0}'.format(num+1)] = imf
    return df_vmfs

In [16]:
def lgbm_scaled_cross_validation_vmf(X, y1,y2,y3,y4,y_true, params):
   # if params == None:
#         model = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
        #model = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor())])
   # else:
#         model = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**params))])
    model1 = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor(**params[0]))])
    model2 = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor(**params[1]))])
    model3 = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor(**params[2]))])
    model4 = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor(**params[3]))])

    print('-----------LGBM CROSS VALIDATION BEGINNING-----------')
    split = 10
    kf = KFold(n_splits=split, shuffle=True)       
    lgbm_rmse_scores = []
    lgbm_mae_scores = []
    i = 1
    for (train_index, test_index) in kf.split(pd.DataFrame(X), pd.DataFrame(y_true)):
        X_train, X_test = pd.DataFrame(X).iloc[train_index], pd.DataFrame(X).iloc[test_index]
        Y1_train, Y1_test = pd.DataFrame(y1).iloc[train_index],pd.DataFrame(y1).iloc[test_index]
        Y2_train, Y2_test = pd.DataFrame(y2).iloc[train_index],pd.DataFrame(y2).iloc[test_index]
        Y3_train, Y3_test = pd.DataFrame(y3).iloc[train_index],pd.DataFrame(y3).iloc[test_index]
        Y4_train, Y4_test = pd.DataFrame(y4).iloc[train_index],pd.DataFrame(y4).iloc[test_index]
        Y_train, Y_test = pd.DataFrame(y_true).iloc[train_index],pd.DataFrame(y_true).iloc[test_index]

        model1.fit(X_train, Y1_train)
        model2.fit(X_train, Y2_train)
        model3.fit(X_train, Y3_train)
        model4.fit(X_train, Y4_train)

        prediction = model1.predict(X_test)+model2.predict(X_test)+model3.predict(X_test)+model4.predict(X_test)
        lgbm_rmse_scores.append(mean_squared_error(Y_test, prediction,squared=False))
        lgbm_mae_scores.append(mean_absolute_error(Y_test, prediction))
        
        print(show_evaluation(prediction, Y_test))
        print(f'-------------------FOLD {i}-----------------')
        i+=1

    print('---------------CROSS VALIDATION COMPLETE-------------')
    print('--------------------------RMSE-----------------------')
    display_scores(lgbm_rmse_scores)
    print('--------------------------MAE------------------------')
    display_scores(lgbm_mae_scores)

## WP1 

| |  | Mean | Std | Sum up |
| --- | --- | --- | --- | --- |
| No params | RMSE | 0.10344875448880764 | 0.0019070131550065564 |  |
| No params - StandardScaler | RMSE | 0.10355855436437653 | 0.0019043784372002228 |  |
| After tuning 50trials| RMSE | 0.06830085723562579 | 0.0012998671387256361 | To keep, maybe redo optuna with warm start with it - first sub|
| After tuning 100trials| RMSE | 0.0695930431604128 | 0.0015123892627707553 | |
| After tuning W.S 60trials | RMSE | 0.0671868765178121 | 0.0019357892229500213 | second sub - BETTER |
| After tuning W.S 50trials - StandardScaler | RMSE | 0.06705406618054667 | 0.0013509038914922506 | with warm start 1 parameters |
| After tuning W.S 50trials - RobustScaler | RMSE | 0.06736753027260156 | 0.0009971394767292064 | |
| --- | --- | --- | --- | --- |
| No params | MAE | 0.07306057409517844 | 0.0009857628815465762 |  |
| No params - StandardScaler | MAE | 0.07310137331348271 | 0.0010945145617042643 |  |
| After tuning 50trials| MAE | 0.04481696427654311 | 0.000727722171899004 |  |
| After tuning 100trials| MAE | 0.045696725242359994 | 0.0008508558100930331 |  |
| After tuning W.S 60trials | MAE | 0.04322865305311156 | 0.0009454253743042544 | |
| After tuning W.S 50trials - StandardScaler | MAE | 0.043173032898780175 | 0.0007696056550296055 | with warm start 1 parameters |
| After tuning W.S 50trials - RobustScaler | MAE | 0.043246198690701294 | 0.000596545751975941 | |

In [87]:
wp1_X = train_wp1[[c for c in train_wp1 if c not in ["wp"]] + ["wp"]].drop(to_drop+feature_corr, axis = 1)
X1 = wp1_X.drop('wp', axis=1)
y1 = wp1_X['wp']

In [ ]:
# lgbm_cross_validation(X1, y1, None)
lgbm_scaled_cross_validation(X1, vmf_1['IMFwp1'], None)

In [ ]:
# lgbm_cross_validation(X1, y1, None)
lgbm_scaled_cross_validation(X1, vmf_1['IMFwp2'], None)

In [ ]:
# lgbm_cross_validation(X1, y1, None)
lgbm_scaled_cross_validation(X1, vmf_1['IMFwp3'], None)

In [ ]:
# lgbm_cross_validation(X1, y1, None)
lgbm_scaled_cross_validation(X1, vmf_1['IMFwp4'], None)

In [88]:
vmf_1 = vmd(y1,4,0,0,0,0,0)

In [27]:
def objective_wp1(trial,data=X1,target=vmf_1['IMFwp4']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp1, n_trials=30)
#write_results('Data/Hyperparametrization/lgbm_vmd_50trials_cv.xlsx', 'wp1', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [89]:
#30
param_1_1 = {
        'reg_alpha': 0.005623861124729257,
   'reg_lambda': 0.22406174588522088,
    'colsample_bytree': 0.7,
 'subsample': 0.4,
 'learning_rate': 0.11907721244377018,
 'max_depth': 31,
 'num_leaves': 776,
 'min_child_samples': 29}
#
param_1_2 = {'reg_alpha': 0.003075982697697176,
 'reg_lambda': 9.876348984550155,
 'colsample_bytree': 0.3,
 'subsample': 0.7,
 'learning_rate': 0.15883690225856262,
 'max_depth': 64,
 'num_leaves': 657,
 'min_child_samples': 20}
#
param_1_3 = {'reg_alpha': 0.018182754684069484,
 'reg_lambda': 0.006831890665560452,
 'colsample_bytree': 0.5,
 'subsample': 0.5,
 'learning_rate': 0.11202987530753453,
 'max_depth': 53,
 'num_leaves': 848,
 'min_child_samples': 27}
    
param_1_4 = {'reg_alpha': 0.0038530142853001356,
 'reg_lambda': 0.0021589196685704304,
 'colsample_bytree': 1.0,
 'subsample': 0.6,
 'learning_rate': 0.05089100428869571,
 'max_depth': 94,
 'num_leaves': 881,
 'min_child_samples': 29}

params_vmf=[param_1_vmf1,param_1_vmf2,param_1_vmf3,param_1_vmf4]

NameError: name 'param_1_vmf1' is not defined

In [33]:
y1_vmfs = []
for i in range(1,5):
    y_vmfs.append(vmf_1['IMFwp'+str(i)])
#y_vmfs

In [41]:
lgbm_scaled_cross_validation_vmf(X1, vmf_1['IMFwp1'],vmf_1['IMFwp2'],vmf_1['IMFwp3'],vmf_1['IMFwp4'],y1, params_vmf)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.06247878471752004
MAE score: 0.04219094218762699
None
-------------------FOLD 1-----------------
RMSE score: 0.06160200834397176
MAE score: 0.04139354072845678
None
-------------------FOLD 2-----------------
RMSE score: 0.06074324030276923
MAE score: 0.04092018458683762
None
-------------------FOLD 3-----------------
RMSE score: 0.0617519139893874
MAE score: 0.04164207713823465
None
-------------------FOLD 4-----------------
RMSE score: 0.06205431054386787
MAE score: 0.04113493309160323
None
-------------------FOLD 5-----------------
RMSE score: 0.060349677858007426
MAE score: 0.04103568153928476
None
-------------------FOLD 6-----------------
RMSE score: 0.06157415042244582
MAE score: 0.04139739465012538
None
-------------------FOLD 7-----------------
RMSE score: 0.0636795056658864
MAE score: 0.042088777860623046
None
-------------------FOLD 8-----------------
RMSE score: 0.06453613020898885
MAE score: 0.0423781923685

In [17]:
# lgbm_cross_validation(X1, y1, params_1)
#lgbm_scaled_cross_validation(X1, y1, params_1)

## WP2

| |  | Mean | Std | Sum up|
| --- | --- | --- | --- | |
| No params | RMSE | 0.10935335541057582 | 0.0014425096116734836 | |
| No params - StandardScaler | RMSE | 0.10938240918068962 | 0.002563571983412213 |  |
| After tuning - 50trials| RMSE | 0.0725081520968898 | 0.0016974702626377217 | |
| After tuning 100trials| RMSE | 0.0707064364904941 | 0.001396820290618349 | More stable, to keep |
| After tuning W.S. 50trials - StandardScaler | RMSE | 0.07072007438762447 | 0.0010396359382112443 | With 100trials parameters |
| After tuning W.S. 50trials - RobustScaler | RMSE | 0.07014346508497502 | 0.0013671602187583412 | |
| 50 trials per vmf - RobustScaler | RMSE | 0.06362570142793085 |0.001287144743120583 | |
| --- | --- | --- | --- |---|
| No params | MAE | 0.07681923856705511 | 0.0008670825615244791 | |
| No params - StandardScaler | MAE | 0.0768829843356125 | 0.0017568758903956435 |  |
| After tuning - 50trials| MAE | 0.04512164110351975 |  0.0006579433030966575 | |
| After tuning 100trials| MAE | 0.04457902842458915 | 0.0006807155447311589 | |
| After tuning W.S. 50trials - StandardScaler | MAE | 0.04456161763710905 | 0.0005342365137369251 | With 100trials parameters |
| After tuning W.S. 50trials - RobustScaler | MAE | 0.044471464183373825 | 0.0007098731360466324 | |
| 50 trials per vmf - RobustScaler | MAE | 0.04180182904890137 |0.000707244027895384 | |

In [18]:
wp2_X = train_wp2[[c for c in train_wp2 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X2 = wp2_X.drop('wp', axis=1)
y2 = wp2_X['wp']

In [19]:
# lgbm_cross_validation(X2, y2, None)
#lgbm_scaled_cross_validation(X2, y2, None)

In [20]:
vmf_2=vmd(y2,4,0,0,0,0,0)

In [21]:
def objective_wp2(trial,data = X2,target = vmf_2['IMFwp1']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp2, n_trials=50)
# write_results('Data/Hyperparametrization/lgbm_100trials.xlsx', 'wp2', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [24]:
def objective_wp2(trial,data = X2,target = vmf_2['IMFwp2']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp2, n_trials=50)
# write_results('Data/Hyperparametrization/lgbm_100trials.xlsx', 'wp2', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [26]:
def objective_wp2(trial,data = X2,target = vmf_2['IMFwp3']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp2, n_trials=50)
# write_results('Data/Hyperparametrization/lgbm_100trials.xlsx', 'wp2', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [28]:
def objective_wp2(trial,data = X2,target = vmf_2['IMFwp4']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp2, n_trials=50)
# write_results('Data/Hyperparametrization/lgbm_100trials.xlsx', 'wp2', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [30]:
param_2_1 ={'reg_alpha': 0.045585103284025534,
 'reg_lambda': 0.009928472317054265,
 'colsample_bytree': 0.7,
 'subsample': 0.5,
 'learning_rate': 0.11386316009595464,
 'max_depth': 40,
 'num_leaves': 824,
 'min_child_samples': 15}


param_2_2 ={'reg_alpha': 0.028080639879528732,
 'reg_lambda': 0.110783705692678,
 'colsample_bytree': 0.8,
 'subsample': 0.5,
 'learning_rate': 0.20799079399613676,
 'max_depth': 70,
 'num_leaves': 517,
 'min_child_samples': 11}


param_2_3 ={'reg_alpha': 0.07246945784504784,
 'reg_lambda': 0.016214373928923043,
 'colsample_bytree': 1.0,
 'subsample': 0.5,
 'learning_rate': 0.17209666016784908,
 'max_depth': 34,
 'num_leaves': 650,
 'min_child_samples': 12}


param_2_4 ={'reg_alpha': 0.07559865044607995,
 'reg_lambda': 0.0011384763043129388,
 'colsample_bytree': 0.7,
 'subsample': 0.4,
 'learning_rate': 0.12470087079296706,
 'max_depth': 99,
 'num_leaves': 977,
 'min_child_samples': 2}


params_2_vmf = [param_2_1,param_2_2,param_2_3,param_2_4]

In [31]:
lgbm_scaled_cross_validation_vmf(X2, vmf_2['IMFwp1'],vmf_2['IMFwp2'],vmf_2['IMFwp3'],vmf_2['IMFwp4'],y2, params_2_vmf)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.06435222767885826
MAE score: 0.04160794282249754
None
-------------------FOLD 1-----------------
RMSE score: 0.06417987832155933
MAE score: 0.042819693509640815
None
-------------------FOLD 2-----------------
RMSE score: 0.06447763896692986
MAE score: 0.04227375382273011
None
-------------------FOLD 3-----------------
RMSE score: 0.06616824987685402
MAE score: 0.04234023806832774
None
-------------------FOLD 4-----------------
RMSE score: 0.06218364973127905
MAE score: 0.04177397988401415
None
-------------------FOLD 5-----------------
RMSE score: 0.06179046386914042
MAE score: 0.04026940043180599
None
-------------------FOLD 6-----------------
RMSE score: 0.06466715692052402
MAE score: 0.042595893456079566
None
-------------------FOLD 7-----------------
RMSE score: 0.06284073054672021
MAE score: 0.041632449894930604
None
-------------------FOLD 8-----------------
RMSE score: 0.0629107599099142
MAE score: 0.04131054696

## WP3

| |  | Mean | Std ||
| --- | --- | --- | --- ||
| No params | RMSE | 0.10392558077951244 | 0.0019038044796542812 ||
| No params - StandardScaler | RMSE | 0.10354114984428979 | 0.0013629826554706927 ||
| After tuning - 50trials| RMSE | 0.058253804820626545 | 0.0009893279354834155 | More stable, to keep |
| After tuning - 100trials| RMSE | 0.058338944346627106 | 0.0017133930174837203 ||
| After tuning W.S. - 50trials - StandardScaler | RMSE | 0.05839355310487706 | 0.0009599824558874801 ||
| After tuning - 50trial - StandardScaler | RMSE | 0.05828701989178382 | 0.0013469437248627486 | with 50trials best|
| After tuning - 50trial W.S. - RobustScaler | RMSE | 0.05907568414783913 | 0.0010120672015320588 | |
| 50 trials per vmf - RobustScaler | RMSE | 0.055222505478173776 |0.001617631570145444 | |
| --- | --- | --- | --- | --- |
| No params | MAE | 0.07550802464973318 | 0.0012006073434917633 ||
| No params - StandardScaler | MAE | 0.0753220326933334 | 0.0007353049410651828 | |
| After tuning - 50trials| MAE | 0.03787310900962521 | 0.000442034368456366 ||
| After tuning - 100trials| MAE | 0.03838030476025398 | 0.0007480100565996748 ||
| After tuning W.S. - 50trials -StandardScaler | MAE | 0.03838277636708219 | 0.0006195357541130345 ||
| After tuning - 50trials - StandardScaler | MAE | 0.03796602386426936 | 0.0006588818520419181 | with 50trials best|
| After tuning - 50trial W.S. - RobustScaler | MAE | 0.03824961618901729 | 0.0007043585347168141 | |
| 50 trials per vmf - RobustScaler | MAE | 0.03743940912757508 |0.0007763774066394942 | |


In [32]:
wp3_X = train_wp3[[c for c in train_wp3 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X3 = wp3_X.drop('wp', axis = 1)
y3 = wp3_X['wp']


In [33]:
vmf_3=vmd(y3,4,0,0,0,0,0)

In [34]:
def objective_wp3(trial,data = X3,target = vmf_3['IMFwp1']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp3, n_trials=50)
# write_results('Data/Hyperparametrization/lgbm_100trials.xlsx', 'wp3', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [36]:
def objective_wp3(trial,data = X3,target = vmf_3['IMFwp2']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp3, n_trials=50)
# write_results('Data/Hyperparametrization/lgbm_100trials.xlsx', 'wp3', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [39]:
def objective_wp3(trial,data = X3,target = vmf_3['IMFwp3']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp3, n_trials=50)
# write_results('Data/Hyperparametrization/lgbm_100trials.xlsx', 'wp3', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [41]:
def objective_wp3(trial,data = X3,target = vmf_3['IMFwp4']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp3, n_trials=50)
# write_results('Data/Hyperparametrization/lgbm_100trials.xlsx', 'wp3', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [43]:
param_3_1={'reg_alpha': 0.09154624212451326,
 'reg_lambda': 3.243003547398231,
 'colsample_bytree': 0.6,
 'subsample': 0.5,
 'learning_rate': 0.08674967041071543,
 'max_depth': 36,
 'num_leaves': 991,
 'min_child_samples': 4}

param_3_2={'reg_alpha': 0.015851789162864068,
 'reg_lambda': 1.1709973762604666,
 'colsample_bytree': 0.5,
 'subsample': 0.8,
 'learning_rate': 0.1264481324316636,
 'max_depth': 64,
 'num_leaves': 754,
 'min_child_samples': 24}

param_3_3={'reg_alpha': 0.009208658021513315,
 'reg_lambda': 0.11295923333720627,
 'colsample_bytree': 0.6,
 'subsample': 0.4,
 'learning_rate': 0.13684811179845982,
 'max_depth': 62,
 'num_leaves': 769,
 'min_child_samples': 16}


param_3_4={'reg_alpha': 0.0043699543264321735,
 'reg_lambda': 0.7966979080149321,
 'colsample_bytree': 0.5,
 'subsample': 0.7,
 'learning_rate': 0.15721415799888475,
 'max_depth': 35,
 'num_leaves': 502,
 'min_child_samples': 1}

params_3_vmf = [param_3_1,param_3_2,param_3_3,param_3_4]

In [44]:
lgbm_scaled_cross_validation_vmf(X3, vmf_3['IMFwp1'],vmf_3['IMFwp2'],vmf_3['IMFwp3'],vmf_3['IMFwp4'],y3, params_3_vmf)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.053138613206103975
MAE score: 0.03644171385000484
None
-------------------FOLD 1-----------------
RMSE score: 0.05511795907919635
MAE score: 0.0376264776465745
None
-------------------FOLD 2-----------------
RMSE score: 0.05530458988968696
MAE score: 0.03789593955017934
None
-------------------FOLD 3-----------------
RMSE score: 0.0543504044736086
MAE score: 0.036521578154908835
None
-------------------FOLD 4-----------------
RMSE score: 0.05582276204889331
MAE score: 0.037871117917446934
None
-------------------FOLD 5-----------------
RMSE score: 0.05670638543572619
MAE score: 0.03796059472508727
None
-------------------FOLD 6-----------------
RMSE score: 0.05295193495600955
MAE score: 0.03629569541249994
None
-------------------FOLD 7-----------------
RMSE score: 0.058660671103486994
MAE score: 0.038746318004650566
None
-------------------FOLD 8-----------------
RMSE score: 0.0542284108890289
MAE score: 0.03703766506

## WP4

| |  | Mean | Std | |
| --- | --- | --- | --- | --- |
| No params | RMSE | 0.10486204816363351 | 0.0015105949978751166 ||
| No params  - Std | RMSE | 0.10480849342496516 | 0.0010370863436755212 ||
| After tuning - 50trials| RMSE | 0.06513233717204232 | 0.0015891617240032727 ||
| After tuning 100trials| RMSE | 0.06357594848470964 | 0.0013676749030776929 ||
| After tuning with W.S - 50trials - Std | RMSE | 0.06339401569270936 | 0.001228053306037005 ||
| After tuning with W.S. - 50trials - RobustScaler | RMSE | 0.0632324115841705 | 0.0010081050240456021 ||
| 50 trials per vmf - RobustScaler | RMSE |0.0593103048767063  |0.0011712884174825636 | |
| --- | --- | --- | --- | --- |
| No params | MAE | 0.07564776733421566 | 0.00104638869825841 ||
| No params  - Std | MAE | 0.07570794104041156 | 0.0008419207475550308 ||
| After tuning - 50trials| MAE | 0.04219236028055372 | 0.0008190579419060266 ||
| After tuning 100trials| MAE |0.04172111697148837  | 0.0009349285385250968 ||
| After tuning with W.S - 50trials  - Std | MAE | 0.04150668920859586 | 0.0005729825500890684 ||
| After tuning with W.S. - 50trials - RobustScaler | MAE | 0.04170428506837879 | 0.0006101247158768171 ||
| 50 trials per vmf - RobustScaler | MAE | 0.04115807160847805 |0.0006471350512019214 | |

In [45]:
wp4_X = train_wp4[[c for c in train_wp4 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X4 = wp4_X.drop('wp', axis = 1)
y4 = wp4_X['wp']

In [46]:
vmf_4 = vmd(y4,4,0,0,0,0,0)

In [47]:
def objective_wp4(trial,data = X4,target = vmf_4['IMFwp1']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp4, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp4', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [49]:
def objective_wp4(trial,data = X4,target = vmf_4['IMFwp2']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp4, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp4', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [51]:
def objective_wp4(trial,data = X4,target = vmf_4['IMFwp3']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp4, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp4', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [53]:
def objective_wp4(trial,data = X4,target = vmf_4['IMFwp4']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp4, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp4', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [56]:
param_4_1={'reg_alpha': 0.01530039580244078,
 'reg_lambda': 1.4574880119337652,
 'colsample_bytree': 0.4,
 'subsample': 0.8,
 'learning_rate': 0.10927203044997952,
 'max_depth': 74,
 'num_leaves': 922,
 'min_child_samples': 5}

param_4_2={'reg_alpha': 0.050090771650523815,
 'reg_lambda': 0.006257336992207459,
 'colsample_bytree': 0.9,
 'subsample': 0.7,
 'learning_rate': 0.1759118344278427,
 'max_depth': 34,
 'num_leaves': 776,
 'min_child_samples': 14}

param_4_3={'reg_alpha': 0.009643730294464633,
 'reg_lambda': 7.295356241344702,
 'colsample_bytree': 0.8,
 'subsample': 1.0,
 'learning_rate': 0.18968035096862862,
 'max_depth': 90,
 'num_leaves': 1000,
 'min_child_samples': 73}

param_4_4={'reg_alpha': 0.015957473793195013,
 'reg_lambda': 1.7448348871971726,
 'colsample_bytree': 0.4,
 'subsample': 0.7,
 'learning_rate': 0.11203629767171021,
 'max_depth': 89,
 'num_leaves': 236,
 'min_child_samples': 1}
params_4_vmf = [param_4_1,param_4_2,param_4_3,param_4_4]

In [57]:
lgbm_scaled_cross_validation_vmf(X4, vmf_4['IMFwp1'],vmf_4['IMFwp2'],vmf_4['IMFwp3'],vmf_4['IMFwp4'],y4, params_4_vmf)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.058602128011655784
MAE score: 0.040974426723734816
None
-------------------FOLD 1-----------------
RMSE score: 0.05721199940328143
MAE score: 0.03978904673934377
None
-------------------FOLD 2-----------------
RMSE score: 0.058716476837484696
MAE score: 0.04095169723167431
None
-------------------FOLD 3-----------------
RMSE score: 0.05942364186626939
MAE score: 0.04095029004922764
None
-------------------FOLD 4-----------------
RMSE score: 0.05918029009762694
MAE score: 0.04119418882178776
None
-------------------FOLD 5-----------------
RMSE score: 0.060148403141174055
MAE score: 0.04138230952019754
None
-------------------FOLD 6-----------------
RMSE score: 0.05890160240984948
MAE score: 0.041387199817516575
None
-------------------FOLD 7-----------------
RMSE score: 0.06087719172054634
MAE score: 0.04210986557718243
None
-------------------FOLD 8-----------------
RMSE score: 0.05858911974330959
MAE score: 0.04069765

## WP5

| |  | Mean | Std |
| --- | --- | --- | --- |
| No params | RMSE | 0.11722129743692011 | 0.0017732599261516583 |
| No params - Std | RMSE | 0.11729790317307003 | 0.0014884067903823003 |
| After tuning - 50trials| RMSE | 0.07721413638593042 | 0.0011020420293213135 |
| After tuning - 100trials| RMSE | 0.07297648991888442 | 0.0014970317509404526 |
| After tuning - 100trials - warm start | RMSE | 0.07362803793800192 | 0.0013223501622953715 |
| After tuning 50trials - std - warm start | RMSE | 0.07379631452164911 | 0.0019249511778190373 |
| After tuning 50trials W.S - RobustScaler | RMSE | 0.07295312559507504 | 0.0017094978756410563 |
| 50 trials per vmf | RMSE | 0.06800612721907516 | 0.001364694216955839 |
| --- | --- | --- | --- |
| No params | MAE | 0.08497074568090211 | 0.0009101526501392155 |
| No params - Std | MAE | 0.0849648687365363 | 0.0011901882563545429 |
| After tuning - 50trials| MAE | 0.051677856581467195 | 0.0006374939894477714 |
| After tuning - 100trials| MAE | 0.04765271414503236 | 0.0006257356756510128 |
| After tuning - 100trials - warm trials | MAE | 0.04785179154681675 | 0.0005795839605605526 |
| After tuning 50trials - std - warm start | MAE | 0.0480147138609328 | 0.0008230908433814974 |
| After tuning 50trials W.S - RobustScaler | MAE | 0.04783683731205745 | 0.0007485320718195094 |
| 50 trials per vmf | MAE | 0.04587332776201404 | 0.0008842958670279714 |

In [58]:
wp5_X = train_wp5[[c for c in train_wp5 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X5 = wp5_X.drop('wp', axis = 1)
y5 = wp5_X['wp']

In [59]:
vmf_5=vmd(y5,4,0,0,0,0,0)

In [60]:
def objective_wp5(trial, data = X5,target = vmf_5['IMFwp1']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp5, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp5', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [62]:
def objective_wp5(trial, data = X5,target = vmf_5['IMFwp2']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp5, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp5', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [64]:
def objective_wp5(trial, data = X5,target = vmf_5['IMFwp3']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp5, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp5', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [66]:
def objective_wp5(trial, data = X5,target = vmf_5['IMFwp4']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp5, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp5', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [68]:
param_5_1={'reg_alpha': 0.0034661042818018295,
 'reg_lambda': 2.9022067856985667,
 'colsample_bytree': 0.7,
 'subsample': 0.5,
 'learning_rate': 0.11393879966016535,
 'max_depth': 62,
 'num_leaves': 506,
 'min_child_samples': 5}

param_5_2={'reg_alpha': 0.015434682291175304,
 'reg_lambda': 7.507396765471906,
 'colsample_bytree': 0.9,
 'subsample': 1.0,
 'learning_rate': 0.19527682029160215,
 'max_depth': 100,
 'num_leaves': 996,
 'min_child_samples': 10}

param_5_3={'reg_alpha': 0.02638276086890781,
 'reg_lambda': 0.2470032961411219,
 'colsample_bytree': 0.7,
 'subsample': 1.0,
 'learning_rate': 0.1159631366255844,
 'max_depth': 76,
 'num_leaves': 943,
 'min_child_samples': 1}

param_5_4={'reg_alpha': 0.04377254363611993,
 'reg_lambda': 4.39351044880846,
 'colsample_bytree': 0.4,
 'subsample': 0.7,
 'learning_rate': 0.07358585964995908,
 'max_depth': 46,
 'num_leaves': 822,
 'min_child_samples': 15}

params_5_vmf = [param_5_1,param_5_2,param_5_3,param_5_4]

In [69]:
lgbm_scaled_cross_validation_vmf(X5, vmf_5['IMFwp1'],vmf_5['IMFwp2'],vmf_5['IMFwp3'],vmf_5['IMFwp4'],y5, params_5_vmf)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.0677073567325694
MAE score: 0.04540471162023237
None
-------------------FOLD 1-----------------
RMSE score: 0.06744174720669406
MAE score: 0.04532773252931264
None
-------------------FOLD 2-----------------
RMSE score: 0.06891569210628036
MAE score: 0.04637317816556597
None
-------------------FOLD 3-----------------
RMSE score: 0.06978729450217731
MAE score: 0.04711666199673882
None
-------------------FOLD 4-----------------
RMSE score: 0.0659621660329265
MAE score: 0.044801029135439846
None
-------------------FOLD 5-----------------
RMSE score: 0.06766523137736596
MAE score: 0.04570656207942218
None
-------------------FOLD 6-----------------
RMSE score: 0.06677912193219068
MAE score: 0.04533530390800361
None
-------------------FOLD 7-----------------
RMSE score: 0.06647969961264442
MAE score: 0.04467676134338677
None
-------------------FOLD 8-----------------
RMSE score: 0.06911087067981043
MAE score: 0.04678517732364

## WP6

| |  | Mean | Std |
| --- | --- | --- | --- |
| No params | RMSE | 0.0940394026188472 | 0.0010749562915831372 |
| No params - std | RMSE | 0.09409110695713666 | 0.0013476052174559326 |
| After tuning - 50trials| RMSE | 0.05404362835213171 | 0.0008595325139047733 |
| After tuning 100trials| RMSE | 0.054861488499908594 | 0.0007335378238383901 |
| After tuning 50trials std - W.S. | RMSE |  0.05446642457662869 | 0.0009850877593637966 |
| After tuning with W.S. - 50trials - RobustScaler | RMSE | 0.053430744425113176 | 0.0010865698281516414 ||
| 50 trials per vmf | RMSE | 0.05055279132726877 | 0.0006313072455945034 |
| --- | --- | --- | --- |
| No params | MAE | 0.070455643271004 | 0.0006641538274191148 |
| After tuning - 50trials| MAE | 0.03657758274248596 | 0.0005325521314198646 |
| After tuning 100trials| MAE | 0.03783933495157941 | 0.00045956939815828987 |
| After tuning 50trials std - warm start| MAE | 0.0367847898055025 | 0.00039001216012464674 |
| After tuning with W.S. - 50trials - RobustScaler | RMSE | 0.03652663713189234 | 0.0006251342721346248 ||
| 50 trials per vmf | MAE | 0.0357185614133554 | 0.00028295806558658535 |

In [70]:
wp6_X = train_wp6[[c for c in train_wp6 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X6 = wp6_X.drop('wp', axis = 1)
y6 = wp6_X['wp']


In [71]:
vmf_6=vmd(y6,4,0,0,0,0,0)

In [72]:
def objective_wp6(trial,data = X6,target = vmf_6['IMFwp1']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp6, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp5', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [74]:
def objective_wp6(trial,data = X6,target = vmf_6['IMFwp2']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp6, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp5', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [76]:
def objective_wp6(trial,data = X6,target = vmf_6['IMFwp3']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp6, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp5', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [78]:
def objective_wp6(trial,data = X6,target = vmf_6['IMFwp4']):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective_wp6, n_trials=50)
# write_results('Data/Hyperparametrization/scaled_lgbm_50trials.xlsx', 'wp5', study.trials_dataframe())
best_trial = study.best_trial.params
best_trial

In [80]:
param_6_1={'reg_alpha': 0.051515195307871846,
 'reg_lambda': 0.03926024162189102,
 'colsample_bytree': 0.5,
 'subsample': 0.6,
 'learning_rate': 0.07848954278195815,
 'max_depth': 57,
 'num_leaves': 868,
 'min_child_samples': 39}

param_6_2={'reg_alpha': 0.13037305839892488,
 'reg_lambda': 1.58383134640443,
 'colsample_bytree': 0.4,
 'subsample': 0.8,
 'learning_rate': 0.1361232166626021,
 'max_depth': 23,
 'num_leaves': 539,
 'min_child_samples': 16}

param_6_3={'reg_alpha': 0.01609775594879223,
 'reg_lambda': 0.00244176632269803,
 'colsample_bytree': 0.8,
 'subsample': 0.8,
 'learning_rate': 0.13915227536127003,
 'max_depth': 21,
 'num_leaves': 646,
 'min_child_samples': 14}

param_6_4={'reg_alpha': 0.08888327814702261,
 'reg_lambda': 0.06078209167150264,
 'colsample_bytree': 0.8,
 'subsample': 1.0,
 'learning_rate': 0.10127033570296538,
 'max_depth': 75,
 'num_leaves': 471,
 'min_child_samples': 4}

params_6_vmf = [param_6_1,param_6_2,param_6_3,param_6_4]

In [81]:
lgbm_scaled_cross_validation_vmf(X6, vmf_6['IMFwp1'],vmf_6['IMFwp2'],vmf_6['IMFwp3'],vmf_6['IMFwp4'],y6, params_6_vmf)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.04949259498664572
MAE score: 0.035567974732241756
None
-------------------FOLD 1-----------------
RMSE score: 0.049974513619843126
MAE score: 0.03516693218255585
None
-------------------FOLD 2-----------------
RMSE score: 0.05183019008176159
MAE score: 0.036241341505116265
None
-------------------FOLD 3-----------------
RMSE score: 0.05067231580700801
MAE score: 0.03572179972979816
None
-------------------FOLD 4-----------------
RMSE score: 0.05133481534936137
MAE score: 0.03612293360739645
None
-------------------FOLD 5-----------------
RMSE score: 0.05009850517099491
MAE score: 0.035590098894691384
None
-------------------FOLD 6-----------------
RMSE score: 0.05058459449218789
MAE score: 0.035619599641763734
None
-------------------FOLD 7-----------------
RMSE score: 0.05065045023033785
MAE score: 0.035652872012177164
None
-------------------FOLD 8-----------------
RMSE score: 0.05056599333368089
MAE score: 0.0357965

# Predictions

## Functions

In [82]:
to_drop_test = ['date','wd','forecast_time', 'forecast', "forecast_dist", 'wp']+feature_corr
def make_prediction_dataset(test, to_drop=to_drop_test):
    test_to_predict = test.dropna(subset=['ws','u','v'], how = 'any') # keeps only lines with u,v,ws,wd
    test_to_predict = test_to_predict[test_to_predict['wp'].isna()] # keeps only lines with no wp
    test_to_predict = test_to_predict.sort_values(by=['date', 'forecast_time'], ascending = [True, False]).drop_duplicates(subset='date')
    test_to_predict = test_to_predict.drop(to_drop, axis = 1)
    return test_to_predict

In [122]:
def make_submission_file(lst_X_trains, lst_y_trains,lst_y_vmfs,lst_tests, lst_models, dates):
    i = 1
    lst_prediction = []
    lst_models_trained = []
    for X, y, test, model in zip(lst_X_trains, lst_y_vmfs, lst_tests, lst_models):
        print(f'--------------Model {i}--------------')
        model[0].fit(X, y[0])
        model[1].fit(X, y[1])
        model[2].fit(X, y[2])
        model[3].fit(X, y[3])
        print(f'True:\n\tMin:{min(lst_y_trains[i-1])}\n\tMax:{max(lst_y_trains[i-1])}\n\tMean:{lst_y_trains[i-1].mean()}')
        predictions = (model[0].predict(test)+
                       model[1].predict(test)+
                       model[2].predict(test)+
                       model[3].predict(test))
        print(f'Prediction:\n\tMin:{min(predictions)}\n\tMax:{max(predictions)}\n\tMean:{np.mean(predictions)}')
#         predictions = [min(y) if i < 0 else i for i in predictions]
#         predictions = [max(y) if i > max(y) else i for i in predictions]
        predictions = [0 if i < 0 else i for i in predictions]
        predictions = [1 if i > 1 else i for i in predictions]
        print(f'Prediction corrected:\n\tMin:{min(predictions)}\n\tMax:{max(predictions)}\n\tMean:{np.mean(predictions)}')
        lst_prediction.append(predictions)
        lst_models_trained.append(model)
        i+=1
    
    df_predictions = pd.DataFrame({
        'date': test_dates,
        'wp1': lst_prediction[0],
        'wp2': lst_prediction[1],
        'wp3': lst_prediction[2],
        'wp4': lst_prediction[3],
        'wp5': lst_prediction[4],
        'wp6': lst_prediction[5],        
    })
    return df_predictions, lst_models_trained

In [123]:
def make_submission_file_nmodels(lst_X_trains, lst_y_trains,lst_y_vmfs,lst_tests, lst_models, dates):
    i = 1
    lst_prediction = []
    lst_models_trained = []
    for X, y, test, model in zip(lst_X_trains, lst_y_vmfs, lst_tests, lst_models):
        print(f'--------------Model {i}--------------')
        for n in range(len(model)):
            model[n].fit(X,y[n])
        
        print(f'True:\n\tMin:{min(lst_y_trains[i-1])}\n\tMax:{max(lst_y_trains[i-1])}\n\tMean:{lst_y_trains[i-1].mean()}')
        
        for m in range(len(model)):
            if m==0:
                predictions = model[m].predict(test)
            else:
                predictions += model[m].predict(test)
        print(f'Prediction:\n\tMin:{min(predictions)}\n\tMax:{max(predictions)}\n\tMean:{np.mean(predictions)}')
#         predictions = [min(y) if i < 0 else i for i in predictions]
#         predictions = [max(y) if i > max(y) else i for i in predictions]
        predictions = [0 if i < 0 else i for i in predictions]
        predictions = [1 if i > 1 else i for i in predictions]
        print(f'Prediction corrected:\n\tMin:{min(predictions)}\n\tMax:{max(predictions)}\n\tMean:{np.mean(predictions)}')
        lst_prediction.append(predictions)
        lst_models_trained.append(model)
        i+=1
    
    df_predictions = pd.DataFrame({
        'date': test_dates,
        'wp1': lst_prediction[0],
        'wp2': lst_prediction[1],
        'wp3': lst_prediction[2],
        'wp4': lst_prediction[3],
        'wp5': lst_prediction[4],
        'wp6': lst_prediction[5],        
    })
    return df_predictions, lst_models_trained

## Submission 

In [91]:
model_1_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_1_1))])
model_1_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_1_2))])
model_1_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_1_3))])
model_1_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_1_4))])
model_1=[model_1_1, model_1_2, model_1_3, model_1_4]
model_2_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_2_1))])
model_2_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_2_2))])
model_2_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_2_3))])
model_2_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_2_4))])
model_2=[model_2_1, model_2_2, model_2_3, model_2_4]
model_3_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_3_1))])
model_3_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_3_2))])
model_3_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_3_3))])
model_3_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_3_4))])
model_3=[model_3_1, model_3_2, model_3_3, model_3_4]
model_4_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_4_1))])
model_4_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_4_2))])
model_4_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_4_3))])
model_4_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_4_4))])
model_4=[model_4_1, model_4_2, model_4_3, model_4_4]
model_5_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_5_1))])
model_5_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_5_2))])
model_5_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_5_3))])
model_5_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_5_4))])
model_5=[model_5_1, model_5_2, model_5_3, model_5_4]
model_6_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_6_1))])
model_6_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_6_2))])
model_6_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_6_3))])
model_6_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**param_6_4))])
model_6=[model_6_1, model_6_2, model_6_3, model_6_4]

In [107]:
model_1_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_1_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_1_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_1_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_1=[model_1_1, model_1_2, model_1_3, model_1_4]
model_2_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_2_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_2_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_2_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_2=[model_2_1, model_2_2, model_2_3, model_2_4]
model_3_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_3_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_3_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_3_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_3=[model_3_1, model_3_2, model_3_3, model_3_4]
model_4_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_4_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_4_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_4_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_4=[model_4_1, model_4_2, model_4_3, model_4_4]
model_5_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_5_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_5_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_5_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_5=[model_5_1, model_5_2, model_5_3, model_5_4]
model_6_1 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_6_2 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_6_3 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_6_4 = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
model_6=[model_6_1, model_6_2, model_6_3, model_6_4]

In [142]:
def vmd(y,k,alpha, tau,DC, init, tol):
    
    #Intrinsic mode generation
     #Empirical Mode Decomposition
    #. some sample parameters for VMD  
   
    u, u_hat, omega = VMD(y,alpha, tau, k, DC, init, tol)
    df_vmfs = pd.DataFrame()
    #Integration in the dataframe
    for num, imf in enumerate(u):
        #print('----Creating VMFwp{0} EMD columns----'.format(num+1))
        df_vmfs['IMFwp{0}'.format(num+1)] = imf
    return df_vmfs

In [204]:
alpha = 1       # moderate bandwidth constraint  
tau = 0          # noise-tolerance (no strict fidelity enforcement)  
k = 4              # k modes  
DC = 0             # no DC part imposed  
init = 1           # initialize omegas randomly 
tol = 1e-6
vmf_1=vmd(y1,k,alpha, tau,DC, init, tol)
vmf_2=vmd(y2,k,alpha, tau,DC, init, tol)
vmf_3=vmd(y3,k,alpha, tau,DC, init, tol)
vmf_4=vmd(y4,k,alpha, tau,DC, init, tol)
vmf_5=vmd(y5,k,alpha, tau,DC, init, tol)
vmf_6=vmd(y6,k,alpha, tau,DC, init, tol)

In [244]:
n_estimators = 50
model_1=[]
model_2=[]
model_3=[]
model_4=[]
model_5=[]
model_6=[]
for i in range(k):
    model=Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(num_iterations=n_estimators))])
    model_1+=[model]
    model_2+=[model]
    model_3+=[model]
    model_4+=[model]
    model_5+=[model]
    model_6+=[model]

In [245]:
y_vmf1=[]
y_vmf2=[]
y_vmf3=[]
y_vmf4=[]
y_vmf5=[]
y_vmf6=[]
for i in range(1,k+1):
    y_vmf1.append(vmf_1['IMFwp'+str(i)])
    y_vmf2.append(vmf_2['IMFwp'+str(i)])
    y_vmf3.append(vmf_3['IMFwp'+str(i)])
    y_vmf4.append(vmf_4['IMFwp'+str(i)])
    y_vmf5.append(vmf_5['IMFwp'+str(i)])
    y_vmf6.append(vmf_6['IMFwp'+str(i)])

In [246]:
# model_1 = LGBMRegressor(**params_1)
# model_2 = LGBMRegressor(**params_2)
# model_3 = LGBMRegressor(**params_3)
# model_4 = LGBMRegressor(**params_4)
# model_5 = LGBMRegressor(**params_5)
# model_6 = LGBMRegressor(**params_6)

lst_models = [model_1, model_2, model_3, model_4, model_5, model_6]
#lst_params_none=[ , , , ]
#lst_models=[lst_params_none,lst_params_none,lst_params_none,lst_params_none,lst_params_none,lst_params_none]
lst_X_trains = [X1, X2, X3, X4, X5, X6]
lst_y_trains = [y1, y2, y3, y4, y5, y6]
lst_y_vmf=[y_vmf1, y_vmf2, y_vmf3, y_vmf4, y_vmf5, y_vmf6]

In [247]:
lst_tests = []
for test in [test_wp1, test_wp2, test_wp3, test_wp4, test_wp5, test_wp6]:
    test = make_prediction_dataset(test)
    lst_tests.append(test)

In [248]:
#df_predictions, lst_models_trained = make_submission_file(lst_X_trains, lst_y_trains, lst_y_vmf,lst_tests, lst_models, test_dates)

In [249]:
df_predictions, lst_models_trained = make_submission_file_nmodels(lst_X_trains, lst_y_trains, lst_y_vmf,lst_tests, lst_models, test_dates)

--------------Model 1--------------
True:
	Min:0.0
	Max:0.96
	Mean:0.2845981952075702
Prediction:
	Min:-0.027970308356061187
	Max:0.9891221566016872
	Mean:0.2994843806165412
Prediction corrected:
	Min:0
	Max:0.9891221566016872
	Mean:0.2995193926117491
--------------Model 2--------------
True:
	Min:0.0
	Max:0.966
	Mean:0.25890153769841273
Prediction:
	Min:-0.039105545128389804
	Max:1.0583572350399826
	Mean:0.25129887543327006
Prediction corrected:
	Min:0
	Max:1
	Mean:0.2513070621840327
--------------Model 3--------------
True:
	Min:0.0
	Max:0.989
	Mean:0.2625247252747253
Prediction:
	Min:-0.03049342198583596
	Max:0.9700561985700036
	Mean:0.2922949445663781
Prediction corrected:
	Min:0
	Max:0.9700561985700036
	Mean:0.29235550234899166
--------------Model 4--------------
True:
	Min:0.0
	Max:0.992
	Mean:0.2763637820512821
Prediction:
	Min:-0.030183240720419217
	Max:0.9707391187601604
	Mean:0.2827602302246812
Prediction corrected:
	Min:0
	Max:0.9707391187601604
	Mean:0.2828100830857916
----

## Saving models

In [250]:
nb_sub = 39
model = "robustscaler-lgbm-vmf"
prepro = 'RobustScaler'
postpro = "Prediction limited by 0-1"

In [251]:
# df_predictions.to_csv('Predictions/submission_nb_10_full_maxabs-lgbm-featselect.csv', index=False, sep=';')
df_predictions.to_csv(f'Predictions/submission_nb_{nb_sub}_{model}.csv', index=False, sep=';')

In [252]:
df_predictions.head()

,date,wp1,wp2,wp3,wp4,wp5,wp6
0,2011010101,0.629442,0.452856,0.075060,0.610177,0.715494,0.616345
1,2011010102,0.587563,0.459752,0.110097,0.578175,0.728987,0.620321
2,2011010103,0.604269,0.417637,0.130435,0.579207,0.766609,0.628782
3,2011010104,0.644538,0.366657,0.279766,0.625416,0.738293,0.627693
4,2011010105,0.668218,0.281546,0.472776,0.717688,0.733243,0.639035
